In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pyloras import LORAS # Import LORA
from sklearn.neighbors import KNeighborsClassifier, KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier  # Already included in RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load your dataset
data = pd.read_csv('/content/dataset-feature-envy.csv')

# Handle missing values (optional)
data = data.dropna()  # Remove rows with missing values (consider imputation if necessary)

data = data.drop(columns=['column_to_drop'], errors='ignore')

# Select only numerical columns
numerical_data = data.select_dtypes(include=['int', 'float'])
data = numerical_data

# Separate features (X) and target variable (y)
y = data['severity']  # Assuming 'severity' is your target column
X = data.drop('severity', axis=1)  # Drop 'severity' from features

# Class distribution before SMOTE (optional)
class_distribution_before = y.value_counts().sort_values(ascending=False)
print("Class Distribution Before Oversampling:\n", class_distribution_before)

# Apply LORAS for oversampling (assuming imbalanced data)
oversample = LORAS()
X_resampled, y_resampled = oversample.fit_resample(X, y)

# Class distribution after oversampling (optional)
class_distribution_after = y_resampled.value_counts().sort_values(ascending=False)
print("Class Distribution After Oversampling:\n", class_distribution_after)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

lda = LinearDiscriminantAnalysis()
X_train_reduced = lda.fit_transform(X_train, y_train)
X_test_reduced = lda.transform(X_test)

# Print the number of features remaining after LDA
num_features_after_lda = X_train_reduced.shape[1]
print(f"Number of features remaining after LDA: {num_features_after_lda}")

# Define and train models
models = {
    "KNN (3 neighbors)": KNeighborsClassifier(n_neighbors=3),
    "KNN (5 neighbors)": KNeighborsClassifier(n_neighbors=5),
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "SVM": SVC(probability=True),  # Enable probability estimates for metrics
    "Logistic Regression": LogisticRegression(),
    "Naive Bayes": GaussianNB(),
}

for name, model in models.items():
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")

    # Classification report (precision, recall, F1-score for each class)
    print(f"{name} Classification Report:\n{classification_report(y_test, y_pred)}")

    # Confusion matrix
    print(f"{name} Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")


Class Distribution Before Oversampling:
 severity
1    280
3     95
2     23
4     22
Name: count, dtype: int64
Class Distribution After Oversampling:
 severity
1    280
2    280
3    280
4    280
Name: count, dtype: int64
Number of features remaining after LDA: 3
KNN (3 neighbors) Accuracy: 0.8571
KNN (3 neighbors) Classification Report:
              precision    recall  f1-score   support

           1       0.72      0.88      0.79        58
           2       0.96      0.88      0.92        57
           3       0.84      0.74      0.79        62
           4       0.98      0.96      0.97        47

    accuracy                           0.86       224
   macro avg       0.87      0.86      0.87       224
weighted avg       0.87      0.86      0.86       224

KNN (3 neighbors) Confusion Matrix:
[[51  1  6  0]
 [ 5 50  2  0]
 [14  1 46  1]
 [ 1  0  1 45]]
KNN (5 neighbors) Accuracy: 0.8125
KNN (5 neighbors) Classification Report:
              precision    recall  f1-score   suppo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Gradient Boosting Accuracy: 0.9821
Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           1       1.00      0.98      0.99        58
           2       1.00      0.96      0.98        57
           3       0.95      0.98      0.97        62
           4       0.98      1.00      0.99        47

    accuracy                           0.98       224
   macro avg       0.98      0.98      0.98       224
weighted avg       0.98      0.98      0.98       224

Gradient Boosting Confusion Matrix:
[[57  0  1  0]
 [ 0 55  2  0]
 [ 0  0 61  1]
 [ 0  0  0 47]]
SVM Accuracy: 0.3884
SVM Classification Report:
              precision    recall  f1-score   support

           1       0.50      0.59      0.54        58
           2       0.42      0.26      0.32        57
           3       0.00      0.00      0.00        62
           4       0.32      0.81      0.46        47

    accuracy                           0.39       224
   macro avg      

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
! pip install -U pyloras